# Imports

In [12]:
import numpy as np
from functools import partial

In [13]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [14]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.backends import cudnn

In [15]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [16]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model

In [17]:
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from sacred import Ingredient

In [18]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset
from utils.metrics import *

In [86]:
from utils import pickle_load, pickle_save
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [20]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [21]:
feature_name = 'r50_gldv2'
set_name = 'train'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [22]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [23]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [24]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [25]:
gnd = pickle_load(osp.join(train_data_dir, gnd_name))

In [26]:
len(gnd['imlist'])

42678

In [27]:
gnd['simlist'][0], len(gnd['simlist'][0])

(['512px-Apollo_Victoria_Theatre.jpg',
  '512px-Margaret_Hamilton_1966.jpg',
  '512px-Hairspray_the_Musical_2012_Aberystwyth_Arts_Centre_Summer_Season_production_poster.png',
  '512px-Beauty_and_Beast.jpg',
  '512px-Use_of_Wondering.jpg',
  '512px-The_Sound_of_Music_Christopher_Plummer_and_Julie_Andrews.jpg',
  '512px-Guys_and_Dolls_(1978).jpg',
  '512px-LauraBellBundyApr10.jpg',
  '512px-Chicago3_(MdB).jpg',
  '512px-Alice_Ghostley_Cousin_Alice_Mayberry_RFD_1970.JPG',
  '512px-Delrae_Knutson_as_Eliza_Doolittle_c1986.jpg',
  '512px-Glinda.png',
  '512px-Helen_Westley.jpg',
  '512px-Alexander_Mann_-_Soubrette.jpg',
  '512px-Mary_Poppins_Returns_Logo_Black.svg.png',
  '512px-Miss_Alice_May_H6724_(cropped).jpg',
  '512px-Jellia_Jamb.jpg',
  '512px-Laurie_Beechman_1984.jpg',
  '512px-Silvi_Vrait_1989.jpg',
  '512px-Dorothy_Loudon.jpg',
  '512px-JLT2017.jpg',
  '512px-Patty_Duke_1975.JPG',
  '512px-South_Pacific_scene.jpg',
  '512px-Stephanie_Kurtzuba_at_the_premiere_of_Bad_Education_(48810

In [28]:
gnd['gnd'][1].keys()

dict_keys(['easy', 'hard', 'junk', 'neg', 'provenance_entity', 'ir_order', 'img_rank_dict', 'rank_img_dict', 'r_easy', 'r_hard', 'r_junk', 'r_neg', 'r_ir_order', 'anchor_idx', 'g_easy', 'g_hard', 'g_junk', 'g_neg'])

In [29]:
def format_selection(selection, order=True):
    
    def preserve_order(array):
        new_array = []
        for e in array:
            if e in new_array:
                continue
            else:
                new_array.append(e)
        return new_array
    
    if order:
        return [preserve_order(sub) for sub in selection]
    else:
        return [list(set(sub)) for sub in selection]

In [30]:
sizes = [len(gnd['simlist'][i]) for i in range(len(gnd['simlist']))]

In [31]:
nn_inds, _ = np.meshgrid(np.arange(100), np.arange(len(gnd['qimlist'])))
nn_inds.shape, nn_inds

((1190, 100),
 array([[ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        ...,
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99]]))

In [32]:
gnd.keys(), gnd['gnd'][1].keys()

(dict_keys(['qimlist', 'imlist', 'simlist', 'is_human', 'gnd']),
 dict_keys(['easy', 'hard', 'junk', 'neg', 'provenance_entity', 'ir_order', 'img_rank_dict', 'rank_img_dict', 'r_easy', 'r_hard', 'r_junk', 'r_neg', 'r_ir_order', 'anchor_idx', 'g_easy', 'g_hard', 'g_junk', 'g_neg']))

In [33]:
for i in range(len(gnd['gnd'])):
    if len(gnd['gnd'][i]['r_easy']) != len(gnd['gnd'][i]['g_easy']):
        print(i)

In [34]:
[x+1 if x >= 45 else x+5 for x in [54,4,6,59]]

[55, 9, 11, 60]

In [35]:
def find_and_replace(array, old_value, new_value):
    return [element if element != old_value else new_value for element in array]

In [36]:
find_and_replace([54,4,6,59], 69, 12)

[54, 4, 6, 59]

In [81]:
def format_gnd_for_training(old_gnd, subset=False):
    
    gnd = []
    
    query_names       = []
    selection_names   = []
    gallery_names     = old_gnd['imlist']
    is_human          = []
    
    for i in range(len(old_gnd['qimlist'])):
        current_query_name = old_gnd['qimlist'][i]
        query_names.append(current_query_name)
        
        current_selection_names = old_gnd['simlist'][i]
        selection_names.append(current_selection_names)
        
        if not subset:
            is_human.append(old_gnd['is_human'][i])
        
        question_gnd = {}
        question_gnd['easy'] = old_gnd['gnd'][i]['easy']
        question_gnd['hard'] = old_gnd['gnd'][i]['hard']
        question_gnd['junk'] = old_gnd['gnd'][i]['junk']
        question_gnd['neg']  = old_gnd['gnd'][i]['neg']

        question_gnd['g_easy'] = old_gnd['gnd'][i]['g_easy']
        question_gnd['g_hard'] = old_gnd['gnd'][i]['g_hard']
        question_gnd['g_junk'] = old_gnd['gnd'][i]['g_junk']
        question_gnd['g_neg']  = old_gnd['gnd'][i]['g_neg']

        question_gnd['anchor_idx']  = old_gnd['gnd'][i]['anchor_idx']

        gnd.append(question_gnd)
        
        for j in range(len(old_gnd['gnd'][i]['easy'])):
            current_positive_image = old_gnd['gnd'][i]['easy'][j]
            if current_positive_image ==  current_query_name:
                continue
            
            query_names.append(current_positive_image)
            selection_names.append(find_and_replace(current_selection_names,
                                                    current_positive_image,
                                                    current_query_name))
            
            if not subset:
                is_human.append(old_gnd['is_human'][i])
            
            positive_gnd = {}
            positive_gnd['easy'] = find_and_replace(old_gnd['gnd'][i]['easy'],
                                                    current_positive_image, 
                                                    current_query_name)
            positive_gnd['hard'] = old_gnd['gnd'][i]['hard']
            positive_gnd['junk'] = old_gnd['gnd'][i]['junk']
            positive_gnd['neg']  = old_gnd['gnd'][i]['neg']
            
            positive_anchor_idx  = gallery_names.index(current_positive_image)

            positive_gnd['g_easy'] = find_and_replace(old_gnd['gnd'][i]['g_easy'],
                                                      positive_anchor_idx,
                                                      old_gnd['gnd'][i]['anchor_idx'])
            
            positive_gnd['g_hard'] = old_gnd['gnd'][i]['g_hard']
            positive_gnd['g_junk'] = old_gnd['gnd'][i]['g_junk']
            positive_gnd['g_neg']  = old_gnd['gnd'][i]['g_neg']

            positive_gnd['anchor_idx']  = positive_anchor_idx
            
            gnd.append(positive_gnd)
    
    new_gnd = {}
    new_gnd['qimlist'] = query_names
    new_gnd['simlist'] = selection_names
    new_gnd['imlist']  = gallery_names
    new_gnd['gnd']     = gnd

    return new_gnd

In [82]:
train_gnd = format_gnd_for_training(gnd)

In [83]:
train_gnd.keys(), train_gnd['gnd'][1].keys(), len(train_gnd['gnd']), len(train_gnd['qimlist']), len(train_gnd['simlist'])

(dict_keys(['qimlist', 'simlist', 'imlist', 'gnd']),
 dict_keys(['easy', 'hard', 'junk', 'neg', 'g_easy', 'g_hard', 'g_junk', 'g_neg', 'anchor_idx']),
 9002,
 9002,
 9002)

In [101]:
train_gnd            = pickle_load(osp.join(train_data_dir, 'gnd_train.pkl'))
humans_train_gnd     = pickle_load(osp.join(train_data_dir, 'humans_gnd_train.pkl'))
non_humans_train_gnd = pickle_load(osp.join(train_data_dir, 'non_humans_gnd_train.pkl'))

In [102]:
new_train_gnd            = format_gnd_for_training(train_gnd)
new_humans_train_gnd     = format_gnd_for_training(humans_train_gnd, subset=True)
new_non_humans_train_gnd = format_gnd_for_training(non_humans_train_gnd, subset=True)

In [103]:
gnd_file = "/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/training_gnd/gnd_train.pkl"
pickle_save(gnd_file, new_train_gnd)

In [104]:
gnd_file = "/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/training_gnd/humans_gnd_train.pkl"
pickle_save(gnd_file, new_humans_train_gnd)

In [105]:
gnd_file = "/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/training_gnd/non_humans_gnd_train.pkl"
pickle_save(gnd_file, new_non_humans_train_gnd)